# Anomalous Financial Transaction Detection

본 대회의 과제는 금융 거래 데이터에서 **이상 거래를 탐지하는 기능**을 개선하고 활용도를 높이는 분류 AI모델을 개발하는 것입니다. 

특히, 클래스 불균형 문제를 해결하기 위해 오픈소스 생성형 AI 모델을 활용하여 부족한 클래스의 데이터를 보완하고, 이를 통해 분류 모델의 성능을 향상시키는 것이 핵심 목표입니다. 

이러한 접근을 통해 금융보안에 특화된 데이터 분석 및 활용 역량을 강화하여 전문 인력을 양성하고, 금융권의 AI 활용 어려움에 따른 해결 방안을 함께 모색하며 금융 산업의 AI 활용 활성화를 지원하는 것을 목표로 합니다.

# Import Library

In [1]:
# 제출 파일 생성 관련
import os
import zipfile

# 데이터 처리 및 분석
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm

# 머신러닝 전처리
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

# 머신러닝 모델
import xgboost as xgb

# 합성 데이터 생성
from sdv.metadata import SingleTableMetadata
from sdv.single_table import CTGANSynthesizer

# To ignore all warnings
import warnings
warnings.filterwarnings('ignore')

# 생성 🏭

# Load Data

In [2]:
train_all = pd.read_csv("/workspace/Dataset/FSI/train.csv")
test_all = pd.read_csv("/workspace/Dataset/FSI/test.csv")

In [3]:
train = train_all.drop(columns="ID")

In [4]:
train["Fraud_Type"].value_counts()

Fraud_Type
m    118800
a       100
j       100
h       100
k       100
c       100
g       100
i       100
b       100
f       100
d       100
e       100
l       100
Name: count, dtype: int64

In [5]:
'''
(*) 리더보드 산식 중 생성데이터의 익명성(TCAP)채점을 위해 각 클래스 별로 1000개의 생성데이터가 반드시 필요합니다.
(*) 본 베이스 라인에서는 "Fraud_Type" 13종류에 대해 1000개씩 , 총 13,000개의 데이터를 생성할 예정입니다.
(*) 분류 모델 성능 개선을 위해 생성 데이터를 활용하는 것에는 생성 데이터의 Row 개수에 제한이 없습니다. 단, 리더보드 평가를 위해 제출을 하는 생성 데이터 프레임은 익명성(TCAP) 평가를 위함이며, 위의 조건을 갖춘 생성 데이터를 제출해야합니다.
'''
N_CLS_PER_GEN = 1000


In [6]:

# 이상치 처리 함수
def handle_outliers(series, n_std=3):
    mean = series.mean()
    std = series.std()
    z_scores = np.abs(stats.zscore(series))
    return series.mask(z_scores > n_std, mean)

# Time_difference 컬럼을 총 초로 변환 및 이상치 처리
train['Time_difference_seconds'] = pd.to_timedelta(train['Time_difference']).dt.total_seconds()
train['Time_difference_seconds'] = handle_outliers(train['Time_difference_seconds'])


# 모든 Fraud_Type 목록 생성 (m 포함)
fraud_types = train['Fraud_Type'].unique()

# 모든 합성 데이터를 저장할 DataFrame 초기화
all_synthetic_data = pd.DataFrame()

N_SAMPLE = 100

# 각 Fraud_Type에 대해 합성 데이터 생성 및 저장
for fraud_type in tqdm(fraud_types):
    
    # 해당 Fraud_Type에 대한 서브셋 생성
    subset = train[train["Fraud_Type"] == fraud_type]

    # 모든 Fraud_Type에 대해 100개씩 샘플링
    subset = subset.sample(n=N_SAMPLE, random_state=42)
    
    # Time_difference 열 제외 (초 단위로 변환된 컬럼만 사용)
    subset = subset.drop('Time_difference', axis=1)
    
    # 메타데이터 생성 및 모델 학습
    metadata = SingleTableMetadata()

    metadata.detect_from_dataframe(subset)
    metadata.set_primary_key(None)

    # 데이터 타입 설정
    column_sdtypes = {
        'Account_initial_balance': 'numerical',
        'Account_balance': 'numerical',
        'Customer_identification_number': 'categorical',  
        'Customer_personal_identifier': 'categorical',
        'Account_account_number': 'categorical',
        'IP_Address': 'ipv4_address',  
        'Location': 'categorical',
        'Recipient_Account_Number': 'categorical',
        'Fraud_Type': 'categorical',
        'Time_difference_seconds': 'numerical',
        'Customer_Birthyear': 'numerical'
    }

    # 각 컬럼에 대해 데이터 타입 설정
    for column, sdtype in column_sdtypes.items():
        metadata.update_column(
            column_name=column,
            sdtype=sdtype
        )
        
    synthesizer = CTGANSynthesizer(
                            metadata,
                            epochs=100
                        )
    synthesizer.fit(subset)

    synthetic_subset = synthesizer.sample(num_rows=N_CLS_PER_GEN)
    
    # 생성된 Time_difference_seconds의 이상치 처리
    synthetic_subset['Time_difference_seconds'] = handle_outliers(synthetic_subset['Time_difference_seconds'])
    
    # Time_difference_seconds를 다시 timedelta로 변환
    synthetic_subset['Time_difference'] = pd.to_timedelta(synthetic_subset['Time_difference_seconds'], unit='s')
    
    # Time_difference_seconds 컬럼 제거
    synthetic_subset = synthetic_subset.drop('Time_difference_seconds', axis=1)
    
    # 생성된 데이터를 all_synthetic_data에 추가
    all_synthetic_data = pd.concat([all_synthetic_data, synthetic_subset], ignore_index=True)
# 최종 결과 확인
print("\nFinal All Synthetic Data Shape:", all_synthetic_data.shape)

100%|██████████| 13/13 [01:17<00:00,  5.94s/it]


Final All Synthetic Data Shape: (13000, 63)


In [7]:
all_synthetic_data.head()

,Customer_Birthyear,Customer_Gender,Customer_personal_identifier,Customer_identification_number,Customer_registration_datetime,Customer_credit_rating,Customer_flag_change_of_authentication_1,Customer_flag_change_of_authentication_2,Customer_flag_change_of_authentication_3,Customer_flag_change_of_authentication_4,...,Last_bank_branch_transaction_datetime,Flag_deposit_more_than_tenMillion,Unused_account_status,Recipient_account_suspend_status,Number_of_transaction_with_the_account,Transaction_history_with_the_account,First_time_iOS_by_vulnerable_user,Fraud_Type,Transaction_resumed_date,Time_difference
0,1988,female,김광수,UXhaKw-xjkJBGo,2005-06-15 17:30:50,A,1,0,1,1,...,2011-07-31 06:32:32,0,1,1,1,0,0,m,2004-07-10 20:11:30,0 days 00:01:38
1,1976,male,엄영식,tLlZuQ-fweLsGp,2005-08-17 23:20:53,C,1,0,0,1,...,2004-07-10 20:11:30,0,1,0,1,2,0,m,2004-07-10 20:11:30,0 days 00:01:38
2,2004,female,김중수,aOYDbp-pOgthCC,2004-07-07 10:19:51,B,1,1,0,1,...,2006-12-17 12:07:09,0,1,0,0,0,0,m,2004-07-10 20:11:30,0 days 00:01:38
3,1980,female,장동현,STcZwr-Uuvgbip,2003-03-12 21:28:24,B,1,1,1,1,...,2004-07-10 20:11:30,1,1,0,2,0,0,m,2013-03-05 04:08:58,0 days 00:01:38
4,1981,male,한준혁,WrnKdP-lsuiVtb,2007-09-20 08:44:38,E,1,0,1,1,...,2004-07-27 17:39:06,0,0,0,0,0,0,m,2004-07-10 20:11:30,0 days 00:01:38


## 원본 데이터와 concat

In [8]:
origin_train = train_all.drop(columns="ID")
train_total = pd.concat([origin_train, all_synthetic_data])
train_total.shape

(133000, 63)

# Data Preprocessing 1 : Select x, y

In [9]:
train_x = train_total.drop(columns=['Fraud_Type'])
train_y = train_total['Fraud_Type']

test_x = test_all.drop(columns=['ID'])

# Data Preprocessing 2 : 범주형 변수 인코딩

In [10]:
le_subclass = LabelEncoder()
train_y_encoded = le_subclass.fit_transform(train_y)

# 변환된 레이블 확인
for i, label in enumerate(le_subclass.classes_):
    print(f"원래 레이블: {label}, 변환된 숫자: {i}")

원래 레이블: a, 변환된 숫자: 0
원래 레이블: b, 변환된 숫자: 1
원래 레이블: c, 변환된 숫자: 2
원래 레이블: d, 변환된 숫자: 3
원래 레이블: e, 변환된 숫자: 4
원래 레이블: f, 변환된 숫자: 5
원래 레이블: g, 변환된 숫자: 6
원래 레이블: h, 변환된 숫자: 7
원래 레이블: i, 변환된 숫자: 8
원래 레이블: j, 변환된 숫자: 9
원래 레이블: k, 변환된 숫자: 10
원래 레이블: l, 변환된 숫자: 11
원래 레이블: m, 변환된 숫자: 12


In [11]:
# train_x
# 'Time_difference' 열을 문자열로 변환
train_x['Time_difference'] = train_x['Time_difference'].astype(str)

# 범주형 변수 인코딩
categorical_columns = train_x.select_dtypes(include=['object', 'category']).columns
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

# 훈련 데이터 인코딩
train_x_encoded = train_x.copy()
train_x_encoded[categorical_columns] = ordinal_encoder.fit_transform(train_x[categorical_columns])

# 특성 순서 저장
feature_order = train_x_encoded.columns.tolist()

# Model Define

In [12]:
from autogluon.tabular import TabularPredictor


train_data = train_x_encoded.copy()
train_data['Fraud_Type'] = train_y_encoded

from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import train_test_split

train_data_split, val_data_split = train_test_split(train_data, test_size=0.2, random_state=42, stratify=train_data['Fraud_Type'])

# Train the model
predictor = TabularPredictor(label='Fraud_Type', eval_metric='accuracy').fit(train_data_split)





No path specified. Models will be saved in: "AutogluonModels/ag-20240824_144150"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Mar 29 23:14:13 UTC 2024
CPU Count:          28
Memory Avail:       19.71 GB / 31.23 GB (63.1%)
Disk Space Avail:   891.27 GB / 1006.85 GB (88.5%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training 

In [13]:
# Make predictions on the validation set
val_predictions = predictor.predict(val_data_split)
val_probabilities = predictor.predict_proba(val_data_split)

# Get true labels
val_true_labels = val_data_split['Fraud_Type']

# Calculate and print loss (accuracy and log loss) by fraud type
fraud_types = val_true_labels.unique()
for fraud_type in fraud_types:
    # Get the indices for the current fraud type
    indices = val_true_labels == fraud_type

    # True labels and predictions for the current fraud type
    true_labels = val_true_labels[indices]
    predictions = val_predictions[indices]
    probabilities = val_probabilities.loc[indices]

    # Calculate accuracy
    accuracy = accuracy_score(true_labels, predictions)
    
    # Handle single-class case for log loss
    if len(true_labels.unique()) == 1:
        print(f"Fraud Type: {fraud_type}")
        print(f" - Accuracy: {accuracy}")
        print(f" - Log Loss: Cannot calculate log loss with only one class present.\n")
    else:
        logloss = log_loss(true_labels, probabilities)
        print(f"Fraud Type: {fraud_type}")
        print(f" - Accuracy: {accuracy}")
        print(f" - Log Loss: {logloss}\n")

Fraud Type: 12
 - Accuracy: 0.9997078464106844
 - Log Loss: Cannot calculate log loss with only one class present.

Fraud Type: 8
 - Accuracy: 0.9318181818181818
 - Log Loss: Cannot calculate log loss with only one class present.

Fraud Type: 3
 - Accuracy: 0.9181818181818182
 - Log Loss: Cannot calculate log loss with only one class present.

Fraud Type: 5
 - Accuracy: 0.9681818181818181
 - Log Loss: Cannot calculate log loss with only one class present.

Fraud Type: 10
 - Accuracy: 0.9681818181818181
 - Log Loss: Cannot calculate log loss with only one class present.

Fraud Type: 2
 - Accuracy: 0.9272727272727272
 - Log Loss: Cannot calculate log loss with only one class present.

Fraud Type: 0
 - Accuracy: 0.9863636363636363
 - Log Loss: Cannot calculate log loss with only one class present.

Fraud Type: 9
 - Accuracy: 0.9363636363636364
 - Log Loss: Cannot calculate log loss with only one class present.

Fraud Type: 7
 - Accuracy: 0.9318181818181818
 - Log Loss: Cannot calculate lo

In [ ]:
# 테스트 데이터 인코딩
test_x_encoded = test_x.copy()
test_x_encoded[categorical_columns] = ordinal_encoder.transform(test_x[categorical_columns])


# 특성 순서 맞추기 및 데이터 타입 일치
test_x_encoded = test_x_encoded[feature_order]
for col in feature_order:
    test_x_encoded[col] = test_x_encoded[col].astype(train_x_encoded[col].dtype)

In [ ]:
# Predict on test data
predictions = predictor.predict(test_x_encoded)

# Reverse transform to get original labels if necessary
predictions_label = le_subclass.inverse_transform(predictions)

# Submission

In [ ]:
# 분류 예측 결과 제출 데이터프레임(DataFrame)
# 분류 예측 결과 데이터프레임 파일명을 반드시 clf_submission.csv 로 지정해야합니다.
clf_submission = pd.read_csv("/workspace/Dataset/FSI/sample_submission.csv")
clf_submission["Fraud_Type"] = predictions_label
clf_submission.head()

,ID,Fraud_Type
0,TEST_000000,m
1,TEST_000001,m
2,TEST_000002,m
3,TEST_000003,m
4,TEST_000004,c


In [ ]:
# 합성 데이터 생성 결과 제출 데이터프레임(DataFrame)
# 합성 데이터 생성 결과 데이터프레임 파일명을 반드시 syn_submission.csv 로 지정해야합니다.
all_synthetic_data.head()

,Customer_Birthyear,Customer_Gender,Customer_personal_identifier,Customer_identification_number,Customer_registration_datetime,Customer_credit_rating,Customer_flag_change_of_authentication_1,Customer_flag_change_of_authentication_2,Customer_flag_change_of_authentication_3,Customer_flag_change_of_authentication_4,...,Last_bank_branch_transaction_datetime,Flag_deposit_more_than_tenMillion,Unused_account_status,Recipient_account_suspend_status,Number_of_transaction_with_the_account,Transaction_history_with_the_account,First_time_iOS_by_vulnerable_user,Fraud_Type,Transaction_resumed_date,Time_difference
0,1950,male,홍현주,KfGNJu-TCOPGMK,2012-12-10 22:02:43,B,1,0,1,1,...,2007-01-30 00:15:58,1,0,1,0,3,0,m,2011-11-29 20:15:09,2 days 22:32:44
1,1969,male,윤영환,IoyIKq-ZWDBMWF,2007-12-08 03:23:09,C,1,1,1,0,...,2008-05-01 03:48:19,0,0,1,1,2,0,m,2014-05-07 10:49:56,0 days 00:01:38
2,1955,female,박성민,PHfnba-CoTCGQS,2012-12-10 22:02:43,C,0,0,1,1,...,2009-11-10 02:36:49,0,1,1,0,0,0,m,2012-03-29 06:30:05,21 days 14:58:49
3,1950,female,김상현,SxnCZF-TcUfNki,2008-04-06 05:05:10,B,1,1,0,1,...,2010-06-06 12:34:02,0,1,0,0,2,0,m,2027-03-11 17:25:08,0 days 00:01:38
4,1950,female,김민서,iOOpeU-LJWWQoW,2012-12-10 22:02:43,A,1,1,0,1,...,2004-07-10 20:11:30,0,0,1,0,0,0,m,2026-11-02 19:51:34,8 days 07:09:19


In [ ]:
'''
(*) 저장 시 각 파일명을 반드시 확인해주세요.
    1. 분류 예측 결과 데이터프레임 파일명 = clf_submission.csv
    2. 합성 데이터 생성 결과 데이터프레임 파일명 = syn_submission.csv

(*) 제출 파일(zip) 내에 두 개의 데이터프레임이 각각 위의 파일명으로 반드시 존재해야합니다.
(*) 파일명을 일치시키지 않으면 채점이 불가능합니다.
'''

# 폴더 생성 및 작업 디렉토리 변경
os.makedirs('/workspace/Dataset/FSI/baseline+autogluon/submission', exist_ok=True)
os.chdir("/workspace/Dataset/FSI/baseline+autogluon/submission/")

# CSV 파일로 저장
clf_submission.to_csv('/workspace/Dataset/FSI/baseline+autogluon/submission/clf_submission.csv', encoding='UTF-8-sig', index=False)
all_synthetic_data.to_csv('/workspace/Dataset/FSI/baseline+autogluon/submission/syn_submission.csv', encoding='UTF-8-sig', index=False)

# ZIP 파일 생성 및 CSV 파일 추가
with zipfile.ZipFile("/workspace/Dataset/FSI/baseline+autogluon/submission/baseline_submission.zip", 'w') as submission:
    submission.write('clf_submission.csv')
    submission.write('syn_submission.csv')
    
print('Done.')

Done.
